# Policy Simulation for A/B/C using `comments.csv` (sample_id/thread_id/text)

This notebook:
- Loads `../data/comments.csv` with columns:
  - `sample_id`, `thread_id`, `role`, `order_in_thread`, `text`
- Applies three moderation policies (A, B, C) to each `text` using an LLM
- Saves the results to `../results/comments_with_policy_results.csv`

It assumes:
- This notebook is located in `notebooks/`
- There is a `data/` folder next to it containing `comments.csv`
- There is a `.env` file in the project root with `OPENAI_API_KEY=...`


In [1]:
# !pip install pandas tqdm python-dotenv openai

import os
import json
import time
from pathlib import Path

import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
from openai import OpenAI


In [2]:
# Load environment variables (expects .env with OPENAI_API_KEY)
load_dotenv()

CLIENT = OpenAI(api_key=os.getenv("OPENAI_API_KEY", ""))

PROJECT_ROOT = Path("..").resolve()
DATA_DIR = PROJECT_ROOT / "data"
RESULTS_DIR = PROJECT_ROOT / "results"

DATA_DIR.mkdir(parents=True, exist_ok=True)
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

COMMENTS_CSV = DATA_DIR / "comments.csv"
OUTPUT_CSV = RESULTS_DIR / "comments_with_policy_results.csv"

print("Project root:", PROJECT_ROOT)
print("Comments CSV:", COMMENTS_CSV)
print("Output CSV  :", OUTPUT_CSV)


Project root: C:\Users\Gibeom Kim\Desktop\UnderGraduate\3. junior\techno_science_자유게시판_iter2
Comments CSV: C:\Users\Gibeom Kim\Desktop\UnderGraduate\3. junior\techno_science_자유게시판_iter2\data\comments.csv
Output CSV  : C:\Users\Gibeom Kim\Desktop\UnderGraduate\3. junior\techno_science_자유게시판_iter2\results\comments_with_policy_results.csv


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()  # .env 읽기

key = os.getenv("OPENAI_API_KEY")
print("OPENAI_API_KEY =", repr(key))


In [4]:
resp = CLIENT.chat.completions.create(
#    model="gpt-4.1-mini",
    model="gpt-5.1",
    messages=[{"role": "user", "content": "이건 API 키 테스트용 메시지입니다."}],
    temperature=0.0,
)

print(resp.choices[0].message.content)


알려주셔서 감사합니다.  
API 키는 정상적으로 작동하는 것으로 보입니다.


In [5]:
POLICY_A_TEXT = """Policy A: High Protection, Low Tolerance (매너 최우선)

**Main principles (요약)**  
- 사람·집단을 향한 인신공격·비하·혐오는 최대한 차단한다.  
- 다만, 욕설이 전혀 없고 인신공격도 없는 비판·불만은 그대로 허용한다.  
- 욕설·비속어가 있어도 ‘사람’이 아닌 ‘정책·제도·수업·서비스’만 겨냥하면 원칙적으로 허용하되 경고만 붙인다.  
- 모호한 경우에도, 인신공격·혐오 쪽이면 차단, 단순 감정표현·정책비판이면 허용 쪽으로 본다.  

#### BLOCK 기준

다음에 해당하면 차단한다.

1. **직접적인 인신공격 / 비하**
   - 욕설·비속어·비하 표현이 **개인, 특정 교수, 학생, 댓글 작성자, 소수의 식별 가능한 사람**을 겨냥할 때  
   - 예:  
     - “저 교수 진짜 병x임”, “야 너 같은 멍청이는 학교 오지 마라”  
     - “저년 얼굴 진짜 역겨움”, “틀딱들은 다 꺼져라”  

2. **집단에 대한 혐오·비하**
   - 성별, 지역, 인종, 성적지향, 장애, 종교 등 **사회적 집단 전체**를 모욕·비하하는 표현  
   - 예: “여자는 원래 이래서 안 됨”, “전라도는 다 거짓말쟁이”  

3. **위협·신상털이·괴롭힘 선동**
   - 폭력·자해·살해·성폭력 등을 암시하거나 직접적으로 위협  
   - 신상 공개, 특정인에 대한 집단 괴롭힘 유도  
   - 예: “저 새끼 학교 앞에서 기다렸다가 한 번 패자”, “쟤 인스타 여기임, 가서 테러하자”  

4. **지속적·집요한 조롱·모욕**
   - 같은 사람을 향한 반복적 모욕, 괴롭힘 패턴이 뚜렷한 경우  

#### WARN_AND_ALLOW 기준

다음은 **게시를 허용하되, 매너·표현 관련 경고를 붙인다.**

1. **욕설·비속어가 있으나 ‘사람’이 아닌 대상**
   - 욕설이 **정책, 제도, 수업, 과제, 학교, 시스템** 등에만 향할 때  
   - 예:  
     - “이 ㅈ같은 등록금 정책 뭐냐 진짜”  
     - “과제 존나 많아서 미치겠음, 학교가 학생을 사람으로 안 봄”  

2. **강한 감정표현·불만, 다소 거친 표현**
   - “개빡친다”, “진짜 미친 거 아님?” 등 **비인칭적 욕설·감탄사** 중심  
   - 특정 개인을 직접 겨냥하지 않고, 상황·경험에 대한 분노 표현  

3. **공적 사안에 대한 날카로운 비판 + 약한 조롱**
   - 교수·학교·학생회 등의 **정책·행동**을 비판하면서 다소 비꼬는 표현이 섞인 경우  
   - 단, “멍청이 교수”, “저 새끼”처럼 **명시적 인신공격**이 되면 BLOCK  

#### ALLOW 기준

다음은 그대로 허용한다.

1. **욕설·비속어가 없는 비판·불만·후기**
   - “수업 퀄리티가 너무 떨어진다”, “과제가 과도하다고 느꼈다”  

2. **정중하거나 중립적인 의견, 정보 공유, 경험담**  

3. **가벼운 농담·슬랭**  
   - 특정인·집단을 비하하지 않는 선의의 농담, 밈 등  
""".strip()

POLICY_B_TEXT = """Policy B: Balanced (매너와 토론의 균형)

**Main principles (요약)**  
- 직접적인 인신공격·혐오·위협은 확실히 차단한다.  
- 욕설이 있어도, **정책·제도·수업·상황**을 향한 비판이면 토론 가치를 우선해 허용하되 경고한다.  
- 욕설이 없고 인신공격도 없으면 대부분 허용한다.  
- 모호할 때는 “토론·정보 가치가 있는가?”를 보고, 있으면 WARN_AND_ALLOW, 없으면 BLOCK 쪽으로 기운다.  

#### BLOCK 기준

1. **명시적 인신공격 (개인 대상)**
   - 욕설·비속어·모욕이 **특정 개인**에게 직접 향할 때  
   - 예: “너 진짜 병x냐”, “저 교수는 쓰레기 인간이다”  

2. **혐오 발언 (보호집단 대상)**
   - 성별, 인종, 지역, 성적지향, 장애, 종교 등 보호집단 전체를 모욕·비하  
   - 예: “게이들은 다 정신병자다”, “저 지역 사람은 다 도둑놈”  

3. **폭력·자해·성폭력 등 위협 / 신상털이 / 괴롭힘 선동**
   - 예: “죽여버리고 싶다”, “쟤 이름이랑 학번 여기다 올림, 알아서 해라”  

4. **토론 가치 없이 순수한 모욕·조롱만 있는 경우**
   - 내용·근거 없이 “병신들”, “다 꺼져라” 등만 반복  

#### WARN_AND_ALLOW 기준

1. **욕설·비속어가 있으나, 주 대상이 ‘정책·제도·수업·상황’**
   - 예:  
     - “이 수업 과제량 진짜 미쳤다, 학생들 생각 1도 안 함”  
     - “기숙사 규정 개같이 만들어놨네”  

2. **강한 감정표현 + 논지·경험 공유**
   - 분노·좌절을 표현하면서도, **구체적 이유·경험·논리**가 있는 경우  
   - 예: “이번 학기 진짜 최악이다. 과제가 겹쳐서 잠을 못 잔다. 이런 일정은 개선해야 한다.”  

3. **공적 인물·기관에 대한 거친 비판**
   - 교수·학생회·학교 등 **공적 역할**에 대한 비판에서 다소 비꼬거나 날카로운 표현 사용  
   - 단, “저 새끼”, “저년” 등 노골적 인신공격이면 BLOCK  

#### ALLOW 기준

1. **욕설 없이 표현된 비판·토론·질문**
   - 예: “등록금 인상이 불가피했는지 근거가 궁금합니다.”  

2. **정보 제공, 경험 공유, 건설적 제안**  

3. **감정 표현이 있으나 인신공격·혐오·위협이 없는 경우**
   - 예: “이번 시험 너무 실망스러웠다”, “정말 스트레스 받는다”
""".strip()

POLICY_C_TEXT = """Policy C: Minimal Regulation, High Freedom (최소 규제, 표현의 자유 우선)

**Main principles (요약)**  
- 명백한 위협·신상털이·심각한 혐오만 확실히 차단한다.  
- 인신공격·욕설이 있어도, 폭력·신상털이·집단혐오 수준이 아니면 원칙적으로 허용하되 필요시 경고만 한다.  
- 토론·표현의 자유를 최우선으로 두고, 불쾌하더라도 스스로 걸러볼 수 있다고 본다.  
- 모호한 경우에는 BLOCK보다 WARN_AND_ALLOW 또는 ALLOW를 선택한다.  

#### BLOCK 기준

1. **명백한 폭력·자해·성폭력 위협**
   - 특정인 또는 집단에게 **현실적 위해**를 암시하거나 선동  
   - 예: “내일 저 교수 찾아가서 죽여버릴 거다”, “쟤 기숙사 방 번호 아니까 찾아가서 한 번 보자”  

2. **신상털이·표적 괴롭힘 선동**
   - 실명, 학번, 연락처, 계정 등 개인 식별 정보 공개 + 괴롭힘 유도  
   - 예: “이 사람 인스타 @xxx, 가서 욕 좀 해줘라”  

3. **노골적 혐오 발언 (보호집단 대상)**
   - 인종·성별·성적지향·장애·종교·지역 등 보호집단에 대한 **극단적 모욕·비인간화 표현**  
   - 예: “난민은 벌레다, 다 쫓아내야 한다”  

4. **지속적이고 집요한 괴롭힘 패턴**
   - 같은 사람을 반복적으로 지목해 괴롭히는 명백한 패턴  

#### WARN_AND_ALLOW 기준

1. **거친 인신공격·욕설이 있으나, 위협·신상털이·집단혐오까지는 아닌 경우**
   - 예: “저 교수 진짜 개같다”, “너 같은 놈이랑은 말 섞기 싫다”  
   - 불쾌하고 공격적이지만, 현실적 위해나 집단혐오로 이어지지 않는 표현  

2. **강한 조롱·비하가 포함된 논쟁**
   - 토론 중 “말도 안 되는 소리 하지 마라, 수준이 너무 낮다” 등 인신공격성 표현이 섞인 경우  

3. **전반적으로 공격적·무례한 톤**
   - 욕설·비속어가 많고, 읽는 이에게 불편함을 줄 수 있으나, 위 기준의 BLOCK 사유는 아닌 경우  

#### ALLOW 기준

1. **위협·신상털이·집단혐오가 전혀 없는 경우 대부분**
   - 욕설·비속어가 있더라도, 개인 간 감정싸움·불만 표현 수준에 그치는 경우  
   - 예: “이 수업 진짜 ㅈ같다”, “아 너무 빡친다 오늘”  

2. **일반적인 비판·토론·경험 공유**
   - 예: “수업 방식이 나와 잘 안 맞는다”, “등록금 인상은 이해하지만 부담이 크다”  

3. **가벼운 농담·슬랭·친한 사이의 장난**
   - 맥락상 상호 합의된 장난으로 보이고, 심각한 위협·혐오가 없는 경우  
""".strip()

POLICIES = {
    "A": POLICY_A_TEXT,
    "B": POLICY_B_TEXT,
    "C": POLICY_C_TEXT,
}


In [6]:
def build_prompt(comment_text: str) -> str:
    """Build a single prompt that explains policies A/B/C
    and asks the model to output a strict JSON result."""
    prompt = f"""You are a content moderator for a Korean university online forum.
Your job is to apply three different moderation policies (A, B, C)
to the same comment and show how each policy would treat the comment.

Here is the definition of each policy:

[Policy A]
{POLICIES['A']}

[Policy B]
{POLICIES['B']}

[Policy C]
{POLICIES['C']}

You must strictly follow each policy as written, not your personal opinion.

For each policy, you MUST output:
- decision: one of [BLOCK, WARN_AND_ALLOW, ALLOW]
- short_reason: brief explanation in 1-2 sentences focusing on which rule applies.
- tone_score: integer 1-5 (1 = very polite, 5 = extremely aggressive)
- debate_value_score: integer 1-5 (1 = no argument or information, 5 = high contribution to debate)

Here is the comment from the forum (in Korean):

"{comment_text}"

Please respond in the following JSON format only:

{{
  "A": {{
    "decision": "...",
    "short_reason": "...",
    "tone_score": ...,
    "debate_value_score": ...
  }},
  "B": {{
    "decision": "...",
    "short_reason": "...",
    "tone_score": ...,
    "debate_value_score": ...
  }},
  "C": {{
    "decision": "...",
    "short_reason": "...",
    "tone_score": ...,
    "debate_value_score": ...
  }}
}}
"""  # noqa: E501
    return prompt.strip()


In [7]:
def call_llm(prompt: str, max_retries: int = 3, sleep_sec: float = 2.0) -> str:
    """Call the OpenAI Chat Completions API and return the model's text output.

    - Uses `gpt-4.1-mini` as a default (cheap+strong enough for this task).
    - Temperature is fixed at 0.0 for deterministic, policy-following behavior.
    """
    last_error = None
    for attempt in range(1, max_retries + 1):
        try:
            response = CLIENT.chat.completions.create(
                model="gpt-4.1-mini",
                messages=[
                    {"role": "user", "content": prompt}
                ],
                temperature=0.0,
            )
            text = response.choices[0].message.content
            return text.strip()
        except Exception as e:
            last_error = e
            print(f"[call_llm] Error on attempt {attempt}: {e}")
            time.sleep(sleep_sec)

    raise RuntimeError(f"LLM call failed after {max_retries} attempts. Last error: {last_error}")


In [8]:
def parse_policy_result(raw_text: str) -> dict:
    """Parse the LLM's raw text into a JSON dict.

    - Assumes the model tries to output valid JSON.
    - Handles common patterns like ```json ... ``` wrappers.
    """
    raw_text = raw_text.strip()

    # Handle fenced code blocks
    if raw_text.startswith("```"):
        lines = raw_text.splitlines()
        if lines and lines[0].startswith("```"):
            lines = lines[1:]
        if lines and lines[-1].startswith("```"):
            lines = lines[:-1]
        raw_text = "\n".join(lines).strip()
        if raw_text.lower().startswith("json"):
            raw_text = raw_text[4:].strip()

    try:
        return json.loads(raw_text)
    except json.JSONDecodeError:
        try:
            start = raw_text.index("{")
            end = raw_text.rindex("}") + 1
            snippet = raw_text[start:end]
            return json.loads(snippet)
        except Exception as e:
            raise ValueError(f"JSON parsing failed. Raw text:\n{raw_text}\nError: {e}")


In [9]:
df = pd.read_csv(COMMENTS_CSV)

expected_cols = {"sample_id", "thread_id", "role", "order_in_thread", "text"}
if not expected_cols.issubset(df.columns):
    raise ValueError(f"CSV must contain columns: {expected_cols}. Found: {df.columns.tolist()}")

print(f"Loaded {len(df)} rows from comments.csv")
df.head()


Loaded 182 rows from comments.csv


,sample_id,thread_id,role,order_in_thread,text
0,1,1,post,0,얼마나 인기없으면 글이 안올라오냐
1,2,1,comment,1,뭐?
2,3,1,comment,2,?
3,4,2,post,0,그냥 궁금해서 하는 투표 AI 쓰면 안되는 시험/과제에서 솔직히 쓴 적 있다/없다 ...
4,5,2,comment,1,과제는 많이들 ai 조금이라도 써봤을듯


In [10]:
results = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    sample_id = row["sample_id"]
    thread_id = row["thread_id"]
    role = row["role"]
    order_in_thread = row["order_in_thread"]
    text = str(row["text"])

    prompt = build_prompt(text)
    raw_response = call_llm(prompt)
    parsed = parse_policy_result(raw_response)

    for policy_key in ["A", "B", "C"]:
        policy_data = parsed.get(policy_key, {})
        results.append({
            "sample_id": sample_id,
            "thread_id": thread_id,
            "role": role,
            "order_in_thread": order_in_thread,
            "policy": policy_key,
            "decision": policy_data.get("decision"),
            "short_reason": policy_data.get("short_reason"),
            "tone_score": policy_data.get("tone_score"),
            "debate_value_score": policy_data.get("debate_value_score"),
        })

results_df = pd.DataFrame(results)
results_df.head()


  0%|          | 0/182 [00:00<?, ?it/s]

100%|██████████| 182/182 [11:40<00:00,  3.85s/it]


,sample_id,thread_id,role,order_in_thread,policy,decision,short_reason,tone_score,debate_value_score
0,1,1,post,0,A,ALLOW,욕설이나 인신공격이 없고 단순한 비판·불만 표현으로 판단됨.,2,2
1,1,1,post,0,B,ALLOW,욕설·인신공격 없고 토론 가치가 낮지만 허용 기준에 부합함.,2,2
2,1,1,post,0,C,ALLOW,위협·신상털이·혐오가 없고 단순한 불만 표현으로 자유롭게 허용함.,2,2
3,2,1,comment,1,A,ALLOW,욕설·비속어가 없고 인신공격도 없는 단순한 의문 표현으로 비판·불만이 아님.,1,1
4,2,1,comment,1,B,ALLOW,욕설·인신공격이 없고 토론 가치가 낮은 단순 의문 표현이지만 허용됨.,1,1


In [11]:
results_df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
print("Saved results to:", OUTPUT_CSV)


Saved results to: C:\Users\Gibeom Kim\Desktop\UnderGraduate\3. junior\techno_science_자유게시판_iter2\results\comments_with_policy_results.csv
